### Helpers and Imports

In [186]:
import random
import math
import sys
from typing import Tuple
sys.setrecursionlimit(1000000)

def is_prime(n, k):
    # Miller-Rabin primality test
    # https://gist.github.com/Ayrx/5884790
    
    if n == 2:
        return True
    if n % 2 == 0:
        return False

    r, s = 0, n - 1
    while s % 2 == 0:
        r += 1
        s //= 2
    for _ in range(k):
        a = random.randrange(2, n - 1)
        x = pow(a, s, n)
        if x == 1 or x == n - 1:
            continue
        for _ in range(r - 1):
            x = pow(x, 2, n)
            if x == n - 1:
                break
        else:
            return False
    return True


def generate_big_prime(size):
    k = 40
    p = random.randrange(2 ** (size - 1), 2 ** size - 1)
    if p % 2 == 0:
        p += 1
    while not is_prime(p, k):
        p += 2
    return p


def egcd(a: int, b: int) -> Tuple[int, int, int]:
    # https://en.wikibooks.org/wiki/Algorithm_Implementation/Mathematics/Extended_Euclidean_algorithm#Recursive_algorithm_2
    """return (g, x, y) such that a*x + b*y = g = gcd(a, b)"""
    if a == 0:
        return (b, 0, 1)
    else:
        b_div_a, b_mod_a = divmod(b, a)
        g, x, y = egcd(b_mod_a, a)
        return (g, y - b_div_a * x, x)


def modinv(a: int, b: int) -> int:
    # https://en.wikibooks.org/wiki/Algorithm_Implementation/Mathematics/Extended_Euclidean_algorithm#Modular_inverse
    """return x such that (x * a) % b == 1"""
    g, x, _ = egcd(a, b)
    if g != 1:
        raise Exception('gcd(a, b) != 1')
    return x % b


def L(x, n):
    # L function used in the Pallier cryptosystem
    return (x - 1) // n


def legendre(a, p):
    """return the Legendre symbol (a/p)"""
    symbol = pow(a, int((p-1)/2), p)
    if symbol == (p - 1):
        return -1
    else:
        return symbol

# Pallier Cryptosystem

[Wikipedia page](https://en.wikipedia.org/wiki/Paillier_cryptosystem)

[Original paper](https://doi.org/10.1007%2F3-540-48910-X_16)

Security based on the [Decisional Composite Residuosity Assumption](https://en.wikipedia.org/wiki/Decisional_composite_residuosity_assumption).

In [193]:
class Pallier:
    def __init__(self, security_param):
        self.pk, self.__sk = gen(security_param)
        
        
    def gen(security_param):
        # 1. Choose two large prime numbers of equal length
        length = security_param
        p = generate_big_prime(length)
        q = generate_big_prime(length)

        # 2a. Compute n = pq and lambda = lcm(p-1, q-1)    OR
        # 2b. Compute n = pq and lambda = phi(n) = (p - 1) * (q - 1)
        n = p * q
        lambda_ = (p - 1) * (q - 1)

        # 3a. Select random integer g from Z_{n^2}^*    OR
        # 3b. Compute g = n + 1
        g = n + 1

        # 4a. Ensure n divides the order of g and calculate mu = (L(g^lambda mod n^2))^{-1} mod n    OR
        # 4b. Calculate mu = phi(n)^{-1} mod n = lambda^{-1} mod n
        mu = modinv(lambda_, n)

        # Set the public key and secret key
        pk = (n, g)
        sk = (lambda_, mu)

        return pk, sk


    def enc(self, m):
        n, g = self.pk

        # 1. Ensure 0 ≤ m < n
        assert (0 <= m),"Message value negative!"
        assert (m < n),"Message value too large. Must be less than %i." %n

        # 2. Select random r where 0 < r < n
        r = random.randrange(1,n)

        # 3. Compute ciphertext as: c = g^m * r^n mod n^2
        n2 = n**2
        c = pow(g, m, n2) * pow(r, n, n2) % n2

        return c


    def dec(self, c):
        n, g = self.pk
        lambda_, mu = self.__sk
        m = (L(pow(c, lambda_, n**2), n) * mu) % n
        return m

Example of Pallier Cryptosystem in use:

In [194]:
security_param = 30
pallier = Pallier(security_param)

m = random.randrange(0, 2**security_param)
print('Input message =', m, '\n')

c = pallier.enc(m)
print('Cyphertext =', c, '\n')

d = pallier.dec(c)
print('Decrypted message =', m)

Input message = 84661286 

Cyphertext = 57324836210470154292936510316524342 

Decrypted message = 84661286


# Goldwasser-Micali (GM) Cryptosystem

[Wikipedia page](https://en.wikipedia.org/wiki/Goldwasser%E2%80%93Micali_cryptosystem)

[Original paper](https://doi.org/10.1145%2F800070.802212)

Security based on the [Quadratic Residuosity Assumption](https://en.wikipedia.org/wiki/Quadratic_residuosity_problem).

In [183]:
class GoldwasserMicali:
    def __init__(self, security_param):
        self.pk, self.__sk = gen(security_param)

        
    def gen(security_param):
        # 1. Choose two large prime numbers
        length = security_param
        p = generate_big_prime(length)
        q = generate_big_prime(length)
        while p == q:
            q = generate_big_prime(length)

        # 2. Calculate N = pq
        N = p * q

        # 3. Find some non-residue x such that the Legendre symbols satisfy (x/p) = (x/q) = -1
        #    and hence the Jacobi symbol (x/N) = +1
        #    (if N is a Blum integer, x = N - 1 satisfies this)
        x = random.randrange(1, N)
        while legendre(x, p) != -1 or legendre(x, q) != -1:
            x = random.randrange(1, N)

        # Set the public key and secret key
        pk = (x, N)
        sk = (p, q)
        return pk, sk

    
    def enc(self, m):
        x, N = self.pk

        # 1. Encode m as a string of bits (m_1, ... , m_n)
        m = bin(m)[2:]

        # 2. For every bit m_i, generate a random y_i in Z_N^*.
        #    c_i = (y_i)^2 * x^{m_i} mod N
        c = []
        for i in m:
            m_i = int(i)
            y_i = random.randrange(1,N)
            while math.gcd(y_i, N) != 1:
                y_i = random.randrange(1,N)
            c_i = (pow(y_i, 2, N) * pow(x, m_i, N)) % N

            # 3. Ciphertext c = (c_1, ... , c_n)
            c.append(c_i)

        return c

    
    def dec(self, c):
        x, N = self.pk
        p, q = self.__sk

        # 1. For each i, using the prime factorization (p, q), determine whether the value c_i is a quadratic residue;
        #    if so, m_i = 0, otherwise m_i = 1
        #    c_i is a quadratic residue iff the c_i is a quadratic residue mod p and mod q
        m = ''
        for c_i in c:
            if legendre(c_i, p) == 1 and legendre(c_i, q) == 1:
                m += '0'
            else:
                m += '1'

        return int(m, 2)

In [185]:
security_param = 30
gm = GoldwasserMicali(security_param)

m = random.randrange(0, 2**security_param)
print('Input message =', m, '\n')

c = gm.enc(m)
print('Cyphertext =', c, '\n')

d = gm.dec(c)
print('Decrypted message =', m)

Input message = 693357036 

Cyphertext = [73307664217819812, 35155189492050252, 643381332604641040, 445874659300447645, 576868178178609126, 233209385631376726, 216528917591737641, 309063369880532502, 194526227167522070, 305857758868059474, 547431433938589763, 91103771544708217, 205643876196512681, 350237575549201890, 173834763942678777, 148180481083650451, 352487684891134635, 425612116927031997, 469035219974501993, 573063143343933880, 290732591399336563, 281425252359309825, 210957570361470325, 115227945616441618, 608640507592484210, 30194682573433186, 67441351556133644, 309966291244272445, 167451830862136310, 618172690409960608] 

Decrypted message = 693357036
